In [ ]:
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
import matplotlib.pyplot as plt
import pandas as pd
import time
from tensorflow.keras.optimizers import SGD,RMSprop,Adam,Adadelta,Adagrad,Adamax,Nadam,Ftrl
from tensorflow.keras.models import Sequential
import seaborn as sns
%matplotlib inline

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
!pip install tf-nightly

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
BackgroundImages=np.load("drive/My Drive/LArTPCdata/BackgroundImages.npy")
SupernovaImages=np.load("drive/My Drive/LArTPCdata/SupernovaImages.npy")

def preprocess_data(SupernovaImages,BackgroundImages):
  '''function to preprocess data for hyperparameter tuning'''
  #concatenate, shuffle and normalise images. Also create labels for them
  images=np.concatenate((SupernovaImages,BackgroundImages))
  labels=np.concatenate((np.ones(len(SupernovaImages)),np.zeros(len(BackgroundImages))))
  images, labels= shuffle(images, labels, random_state=0)
  images=images.reshape(len(images),48,48,1).astype('float32')
  images/=np.amax(images)

  #take 20% of images for testing
  train_size=np.int(0.8*len(images))
  train_images=images[:train_size]
  train_labels=labels[:train_size]
  test_images=images[train_size:]
  test_labels=labels[train_size:]

  #take 20% of the remaining data for validation
  training_size=np.int(0.8*len(train_images))
  training_images=train_images[:training_size]
  training_labels=train_labels[:training_size]
  validation_images=train_images[training_size:]
  validation_labels=train_labels[training_size:]

  return training_images,training_labels,validation_images,validation_labels,test_images,test_labels

training_images,training_labels,validation_images,validation_labels,test_images,test_labels=preprocess_data(SupernovaImages,BackgroundImages)

In [ ]:
#create a binary (2 classes) and a categorical model (more than 2 classes)

def binary_model():
  model=tf.keras.models.Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation="relu", input_shape=(48,48,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512,activation="relu"),
    Dense(1, activation="sigmoid")
  ])
  return model

def categorical_model():
    model=tf.keras.models.Sequential([
      Conv2D(filters=32, kernel_size=(3,3), activation="relu", input_shape=(48,48,1)),
      MaxPooling2D(2,2),
      Conv2D(64, (3, 3), activation='relu'),
      MaxPooling2D(2,2),
      Conv2D(128, (3, 3), activation='relu'),
      MaxPooling2D(2,2),
      Flatten(),
      Dense(512,activation="relu"),
      Dense(2, activation="softmax")
    ])
    return model

In [ ]:
#define best optimizer and epochs

def test_CNN_opt(opt,EPOCHS,BATCHES):
  '''function to find the validation accuracy of all optimisers, trained with same parameters'''
  model=binary_model()
  #model.summary()
  model.compile(optimizer=opt, loss="binary_crossentropy",metrics=['accuracy'])
  history = model.fit(training_images, training_labels, epochs=EPOCHS, batch_size=BATCHES, validation_data=(validation_images, validation_labels),verbose=0,shuffle=True)

  return history.history['val_accuracy']



lr=0.001
optimizers=[SGD(learning_rate=lr),RMSprop(learning_rate=lr),Adam(learning_rate=lr),Adadelta(learning_rate=lr),Adagrad(learning_rate=lr),Adamax(learning_rate=lr),Nadam(learning_rate=lr),Ftrl(learning_rate=lr)]
labels_opt=["SGD","RMSprop","Adam","Adadelta","Adagrad","Adamax","Nadam","Ftrl"]

EPOCHS=100
BATCHES=32

#plot validation accuracy for optimisers
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('Validation Accuracy for different Optimizers')
ax.set_xlabel('Epoch')
_ = ax.set_ylabel('Validation Accuracy')
plt.ylim(0.8)
x=np.arange(1,EPOCHS+1,1)

#calculate the validation accuracy
for opt in optimizers:
  ax.plot(x,test_CNN_opt(opt,EPOCHS,BATCHES))

plt.legend(title="Optimizer",labels=labels_opt,bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig("drive/My Drive/Plots/models.png",bbox_inches = 'tight');

In [ ]:
#check whether the binary or the sparse model yields better accuracies, using the best optimiser RMSprop, and plot results

EPOCHS= 60

plt.figure()
plt.title('Validation Accuracy for Binary and Sparse RMSprop')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
x=np.arange(1,EPOCHS+1,1)

model=binary_model()
#model.summary()
model.compile(optimizer=RMSprop(learning_rate=lr), loss="binary_crossentropy",metrics=['accuracy'])
history = model.fit(training_images, training_labels, epochs=EPOCHS, batch_size=BATCHES, validation_data=(validation_images, validation_labels),verbose=0,shuffle=True)

plt.plot(x,history.history['val_accuracy'],label="Binary")


model=categorical_model()
#model.summary()
model.compile(optimizer=RMSprop(learning_rate=lr), loss="sparse_categorical_crossentropy",metrics=['accuracy'])
history = model.fit(training_images, training_labels, epochs=EPOCHS, batch_size=BATCHES, validation_data=(validation_images, validation_labels),verbose=0,shuffle=True)

plt.plot(x,history.history['val_accuracy'],label="Sparse")



plt.legend(loc='best')
plt.savefig("drive/My Drive/Plots/modelstype.png",bbox_inches = 'tight');

In [ ]:
#function to test the model for different hyperparameters

def test_CNN(EPOCHS,BATCHES,lr,rho):

  model=binary_model()
  #model.summary()
  model.compile(optimizer=RMSprop(learning_rate=lr,rho=rho), loss="binary_crossentropy",metrics=['accuracy'])
  history = model.fit(training_images, training_labels, epochs=EPOCHS, batch_size=BATCHES, validation_data=(validation_images, validation_labels),verbose=0,shuffle=True)

  return history.history['val_accuracy']

In [ ]:
#find best rho parameter

discounting_factor=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title("Effect of Binary RMSprop's Discounting Factor on Validation Accuracy")
ax.set_xlabel('Epoch')
_ = ax.set_ylabel('Validation Accuracy')
x=np.arange(1,EPOCHS+1,1)

for rho in discounting_factor:
  ax.plot(x,test_CNN(EPOCHS,BATCHES,lr,rho),label=(str(rho)))

plt.legend(title="Discounting Factor",bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig("drive/My Drive/Plots/rho.png",bbox_inches = 'tight');

In [ ]:
rho=0.9 #default value, all have same effect

#find best learning rate

learning_rates=[1,0.1,0.01,0.001,0.0001,0.00001,0.000001]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title("Effect of Binary RMSprop's Learning Rate on Validation Accuracy")
ax.set_xlabel('Epoch')
_ = ax.set_ylabel('Validation Accuracy')
x=np.arange(1,EPOCHS+1,1)

for lr in learning_rates:
  ax.plot(x,test_CNN(EPOCHS,BATCHES,lr,rho),label=(str(lr)))

plt.legend(title="Learning Rate",bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig("drive/My Drive/Plots/lr.png",bbox_inches = 'tight');

In [ ]:
lr=0.001 #default value, all have the same effect

#find best batch size

batch_size=[8,16,32,64,128]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title("Effect of Binary RMSprop's Batch Size on Validation Accuracy")
ax.set_xlabel('Epoch')
_ = ax.set_ylabel('Validation Accuracy')
x=np.arange(1,EPOCHS+1,1)

for bs in batch_size:
  ax.plot(x,test_CNN(EPOCHS,bs,lr,rho),label=(str(bs)))

plt.legend(title="Batch Size",bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig("drive/My Drive/Plots/batch.png",bbox_inches = 'tight');

In [ ]:
#set optimal epochs and batches
EPOCHS=50
BATCHES=64

In [ ]:
class EnergyDeposition():
    '''class to find the energy deposition of each event, in order to bin the data for energy in the next ML implementation'''
    def __init__(self):
        
        # load the neutrino event data produced by GEANT-4 
        self.electron_data    = np.load("drive/My Drive/forGiulio/marley.npy")

        
    def getEnergyDep(self,Nevents):
        curEv=0
        firstIndex=0
        numLines=np.shape(self.electron_data)[0]
        outputArray=np.zeros((Nevents))
        for i in range(numLines):
            if self.electron_data[i,4]!=curEv:
                outputArray[curEv]=self.processEventFast(curEv,firstIndex,i)
                curEv+=1
                firstIndex=i
                if(curEv>=Nevents):
                    break           
        return outputArray
              
    
    def processEventFast(self,event,first,last):

        #just sum the energy depositions
        event_data=self.electron_data[first:last,0:4] #fill x,y,z and edep array
        return np.sum(event_data[:,3])   

e=EnergyDeposition()
edep=e.getEnergyDep(10000)
indices=np.load("drive/My Drive/LArTPCdata/indices.npy")
edep=np.delete(edep,indices,axis=0) #delete the images that previously gave no signal

#plot the energy deposition from MARLEY data
plt.figure()
plt.hist(edep,bins=60,density=True)
plt.title("Energy Distribution of Supernova Events from MARLEY")
plt.xlabel("Energy (MeV)")
plt.ylabel("Frequency")
plt.savefig("drive/My Drive/Plots/edep.png")

#remove a few images to get arrays of same size when splitting the images for testing data (the ML model is run for all the data is testing data individually)
edep=edep[:9660]

In [ ]:
indices1=[]
indices2=[]
indices3=[]
indices4=[]
indices5=[]
indices6=[]

#bin the events from their energy deposition
for i in range(np.int(len(edep)/5)):
  if edep[i]<10:
    indices1.append(i)
  elif edep[i]<15:
    indices2.append(i)
  elif edep[i]<20:
    indices3.append(i)
  elif edep[i]<25:
    indices4.append(i)
  elif edep[i]<30:
    indices5.append(i)
  else:
    indices6.append(i)

edep_index=[indices1,indices2,indices3,indices4,indices5,indices6] #save indices for each bin

In [ ]:
def preprocess_energy(SupernovaImages,BackgroundImages):
  '''function to preprocess data'''
  #concatenate and normalise images. Also create labels for them
  images=np.concatenate((SupernovaImages,BackgroundImages))
  labels=np.concatenate((np.ones(len(SupernovaImages)),np.zeros(len(BackgroundImages))))

  images=images.reshape(len(images),48,48,1).astype('float32')
  images/=np.amax(images)

  #take 20% of images for testing
  test_size=np.int(0.2*len(images))
  train_images=images[test_size:]
  train_labels=labels[test_size:]
  test_images=images[:test_size]
  test_labels=labels[:test_size]

  #take 20% of the remaining data for validation
  training_size=np.int(0.8*len(train_images))
  training_images=train_images[:training_size]
  training_labels=train_labels[:training_size]
  validation_images=train_images[training_size:]
  validation_labels=train_labels[training_size:]

  return training_images,training_labels,validation_images,validation_labels,test_images,test_labels


def run_energy(screen):
  eff_all_lifetimes=[] #store efficiency for each bin for all lifetimes
  lifetime=[3000,2500,2000,1500,1000,800,500,300,200,100]
  #loop over lifetimes
  for l in lifetime:
    larImages=np.load("drive/My Drive/LArTPCdata/SupernovaImagesS"+f'{screen}'+"L"+f'{l}'+".npy")
    BackgroundImages=np.load("drive/My Drive/LArTPCdata/BackgroundImagesS"+f'{screen}'+"L"+f'{l}'+".npy")
    larImages=larImages[:9660]
    BackgroundImages=BackgroundImages[:9660]

    training_images,training_labels,validation_images,validation_labels,test_images,test_labels=preprocess_energy(larImages,BackgroundImages)

    #run model
    model=binary_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy",metrics=['accuracy'])
    model.fit(training_images, training_labels, epochs=EPOCHS, batch_size=BATCHES, validation_data=(validation_images, validation_labels),verbose=0,shuffle=True)

    eff=[] #store efficiencies for each bin for one lifetime
    for indices in edep_index:
        binned_test_images=np.take(test_images,indices,axis=0) #take indices that match the energy bin
        labels_true=np.ones(len(binned_test_images))
        #make prediction
        prediction=model.predict(binned_test_images,verbose=0)[:,0]
        labels_prediction=np.round(prediction)
        #0: negative (background), 1: positive (supernova)
        tn, fp, fn, tp = confusion_matrix(labels_true, labels_prediction, labels=[0,1]).ravel()
        eff.append(tp/len(binned_test_images))

    eff_all_lifetimes.append(eff)

  return eff_all_lifetimes  

In [ ]:
def plot_energy(screen,efficiency):

  fig = plt.figure()
  ax = fig.add_subplot(111)
  #ax.set_title("Efficiency of Prediction for different Energy Depositions")
  ax.set_xlabel('Energy (MeV)')
  _ = ax.set_ylabel('Testing Efficiency')
  x=np.arange(1,7,1)
  labels=["0","<10","10-15","15-20","20-25","25-30",">30"]
  ax.set_xticklabels(labels)
  ax.set_ylim(0.5)
  lifetime=[3000,2500,2000,1500,1000,800,500,300,200,100]
  #colors=['tab:cyan','tab:olive','tab:gray','tab:pink','tab:brown','tab:purple','tab:red','tab:green','tab:orange','tab:blue']
  new_lifetime=[]
  
  count=0
  for eff in efficiency:
    if np.max(eff)>0.5:
      ax.plot(x,eff,"-o")
      new_lifetime.append(lifetime[count])
    count+=1

  plt.legend(title="Lifetime (μs)",labels=[str(l) for l in new_lifetime],bbox_to_anchor=(1.05, 1), loc='upper left')
  plt.savefig("drive/My Drive/Plots/efficiency"+f'{screen}'+".png",bbox_inches = 'tight');

In [ ]:
eff116=run_energy(116)
plot_energy(116,eff116)

In [ ]:
eff1797=run_energy(1797)
plot_energy(1797,eff1797)

In [ ]:
eff3478=run_energy(3478)
plot_energy(3478,eff3478)

In [ ]:
def run_CNN_rate(s,l,r):
  '''function to run the model for different rates in the detector'''
  larImages=np.load("drive/My Drive/LArTPCdata/SupernovaImagesS"+f'{s}'+"L"+f'{l}'+"R"+f'{r}'+".npy")
  BackgroundImages=np.load("drive/My Drive/LArTPCdata/BackgroundImagesS"+f'{s}'+"L"+f'{l}'+"R"+f'{r}'+".npy")

  training_images,training_labels,validation_images,validation_labels,test_images,test_labels=preprocess_data(larImages,BackgroundImages)
    
  model=binary_model()
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy",metrics=['accuracy'])
  model.fit(training_images, training_labels, epochs=EPOCHS, batch_size=BATCHES, validation_data=(validation_images, validation_labels),verbose=0,shuffle=True)

  #make predictions on the data
  prediction=model.predict(test_images,verbose=0)[:,0]
  labels_prediction=np.round(prediction)

  #0: negative (background), 1: positive (supernova)
  tn, fp, fn, tp = confusion_matrix(test_labels, labels_prediction, labels=[0,1]).ravel()
  acc_SN=tp/len(np.where(test_labels==1)[0])
  acc_BG=tn/len(np.where(test_labels==0)[0]) 
  uncert_SN=np.sqrt((acc_SN*(1-acc_SN))/len(np.where(test_labels==1)[0]))
  uncert_BG=np.sqrt((acc_BG*(1-acc_BG))/len(np.where(test_labels==0)[0]))

  return acc_SN,acc_BG,uncert_SN,uncert_BG

In [ ]:
screen=1797
lifetime=1000
rate=[5,10,15,20,25,30,35,40,45,50,60,70,80,90,100,120,150,175,200,250,300,400,500,600,700,800,900,1000]

acc_SN=[]
acc_BG=[]
uncertainty_SN=[]
uncertainty_BG=[]
N_fp=[]

#loop over rate values
for r in rate:
  acc1,acc2,uncert1,uncert2=run_CNN_rate(screen,lifetime,r)

  acc_SN.append(acc1)
  acc_BG.append(acc2)
  uncertainty_SN.append(uncert1)
  uncertainty_BG.append(uncert2)

  #calculate number of false positives
  fp_percentage=1-acc2
  N_fp.append(r*1e6*fp_percentage)

In [ ]:
plt.figure()
#sns.regplot(x=rate, y=acc_SN,label="Supernova")
#sns.regplot(x=rate, y=acc_BG,label="Radiological")
plt.scatter(rate,acc_SN,label="Supernova",color='tab:blue')
plt.scatter(rate,acc_BG,label="Radiological",color="tab:orange")
plt.axhline(np.mean(acc_SN),color='tab:blue')
plt.axhline(np.mean(acc_BG),color="tab:orange")
plt.errorbar(rate,acc_SN, yerr=uncertainty_SN,color='tab:blue',fmt='o',capsize=3)
plt.errorbar(rate,acc_BG, yerr=uncertainty_BG,color='tab:orange',fmt='o',capsize=3)
plt.title("Effect of Radiological Decay Rates on Testing Accuracy")
plt.xlabel('Activity (μBq)')
plt.ylabel('Testing Accuracy')
plt.legend()
plt.tight_layout()
plt.savefig("drive/My Drive/Plots/Radiorates.png",bbox_inches = 'tight');

In [ ]:
def bg_rejection(x):
  '''calculate background rejection required to misclassify a maximum of 100 neutrinos'''
  return 1- 100/(x*1e6)

#plot relative error and runtime against N
color = 'tab:orange'
fig, ax1 =plt.subplots()
ax1.set_title("Number of False Positives per second and required Background Accuracy")
ax1.set_xlabel("Activity (μBq)")
ax1.set_ylabel("Required Background Accuracy",color=color)
ax1.plot(rate,bg_rejection(np.asarray(rate)),color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_yscale('log')

ax2=ax1.twinx() # second axes with same x-axis

color = 'tab:blue'
ax2.set_ylabel("$N_{FP}$",color=color)
ax2.plot(rate,N_fp,"-o",color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yscale('log')

plt.savefig("drive/My Drive/Plots/N_fp.png",bbox_inches = 'tight')
plt.show();

In [ ]:
def preprocess_images(images,labels,max):
  '''function to preprocess data, customised to fit the next machine learning implementation'''
  images,labels=shuffle(images,labels, random_state=0)
  images=images.reshape(len(images),48,48,1).astype('float32')
  images/=max

  return images,labels

def split_images(images,labels):
  train_size=np.int(0.8*len(images))
  train_images=images[:train_size]
  train_labels=labels[:train_size]
  test_images=images[train_size:]
  test_labels=labels[train_size:]

  training_size=np.int(0.8*len(train_images))
  training_images=train_images[:training_size]
  training_labels=train_labels[:training_size]
  validation_images=train_images[training_size:]
  validation_labels=train_labels[training_size:]

  return training_images,training_labels,validation_images,validation_labels,test_images,test_labels



def run_CNN_ratio():
  '''function to run the model for an increased ratio of background images'''

  acc_SN=[]
  uncertainty_SN=[]
  acc_BG=[]
  uncertainty_BG=[]

  SupernovaImages=np.load("drive/My Drive/LArTPCdata/SupernovaImages.npy")
  SupernovaLabels=np.ones(len(SupernovaImages))

  maxSN=np.amax(SupernovaImages)

  SNtrain_i,SNtrain_l,SNval_i,SNval_l,SNtest_i,SNtest_l=split_images(SupernovaImages,SupernovaLabels)


  BackgroundImgs=np.load("drive/My Drive/LArTPCdata/BackgroundImagesratio.npy")

  #loop to take an increasing amount of background images for training
  for i in range(1,11): 
    BackgroundImages=BackgroundImgs[:(np.int(9662*i))]
    
    BackgroundLabels=np.zeros(len(BackgroundImages))

    #find the maximal value for normalisation of all data on the same scale
    maxBG=np.amax(BackgroundImages)
    if maxBG>maxSN:
      max=maxBG
    else:
      max=maxSN

    #preprocess data
    BGtrain_i,BGtrain_l,BGval_i,BGval_l,BGtest_i,BGtest_l=split_images(BackgroundImages,BackgroundLabels)


    training_images=np.concatenate((SNtrain_i,BGtrain_i))
    training_labels=np.concatenate((SNtrain_l,BGtrain_l))
    training_images, training_labels= preprocess_images(training_images,training_labels,max)
    validation_images=np.concatenate((SNval_i,BGval_i))
    validation_labels=np.concatenate((SNval_l,BGval_l))
    validation_images, validation_labels= preprocess_images(validation_images,validation_labels,max)
    test_images=np.concatenate((SNtest_i,BGtest_i))
    test_labels=np.concatenate((SNtest_l,BGtest_l))
    test_images, test_labels= preprocess_images(test_images,test_labels,max)


    #fit the model
    model=binary_model()
    model.compile(optimizer=RMSprop(), loss="binary_crossentropy",metrics=['accuracy'])
    model.fit(training_images, training_labels, epochs=EPOCHS, batch_size=BATCHES, validation_data=(validation_images, validation_labels),verbose=0,shuffle=True)

    #predict
    prediction=model.predict(test_images,verbose=0)[:,0]
    labels_prediction=np.round(prediction)
    #0: negative (background), 1: positive (supernova)
    tn, fp, fn, tp = confusion_matrix(test_labels, labels_prediction, labels=[0,1]).ravel()
    
    #append results to arrays
    acc1=tp/len(np.where(test_labels==1)[0])
    uncert1=np.sqrt((acc1*(1-acc1))/len(np.where(test_labels==1)[0]))

    acc_SN.append(acc1)
    uncertainty_SN.append(uncert1)

    acc2=tn/len(np.where(test_labels==0)[0])
    uncert2=np.sqrt((acc2*(1-acc2))/len(np.where(test_labels==0)[0]))

    acc_BG.append(acc2)
    uncertainty_BG.append(uncert2)

  return acc_SN,uncertainty_SN,acc_BG,uncertainty_BG

In [ ]:
acc_SN,uncertainty_SN,acc_BG,uncertainty_BG=run_CNN_ratio()

In [ ]:
ratio=np.arange(1,11,1)

plt.figure()
#sns.regplot(x=ratio, y=acc_SN,label="Supernova")
plt.scatter(ratio,acc_SN,label="Supernova",color='tab:blue')
plt.axhline(np.mean(acc_SN),color='tab:blue')
plt.errorbar(ratio,acc_SN, yerr=uncertainty_SN,color='tab:blue',fmt='o',capsize=3)
#sns.regplot(x=ratio, y=acc_BG,label="Radiological")
plt.scatter(ratio,acc_BG,label="Radiological",color='tab:orange')
plt.axhline(np.mean(acc_BG),color='tab:orange')
plt.errorbar(ratio,acc_BG, yerr=uncertainty_BG,color='tab:orange',fmt='o',capsize=3)
plt.title("Efficiency against BG/SN training ratio")
plt.xlabel('BG/SN ratio')
plt.ylabel('Testing Efficiency')
plt.legend(loc='best')
plt.savefig("drive/My Drive/Plots/Radioratio.png",bbox_inches = 'tight');

In [ ]:
def preprocess_images(images,labels,max):
  '''function to preprocess data, customised to fit the next machine learning implementation'''
  images,labels=shuffle(images,labels, random_state=0)
  images=images.reshape(len(images),48,48,1).astype('float32')
  images/=max

  return images,labels


def split_images(images,labels):
  train_size=np.int(0.8*len(images))
  train_images=images[:train_size]
  train_labels=labels[:train_size]
  test_images=images[train_size:]
  test_labels=labels[train_size:]

  training_size=np.int(0.8*len(train_images))
  training_images=train_images[:training_size]
  training_labels=train_labels[:training_size]
  validation_images=train_images[training_size:]
  validation_labels=train_labels[training_size:]

  return training_images,training_labels,validation_images,validation_labels,test_images,test_labels

In [ ]:
def run_CNN_rate_2(r):
  '''function to run the model for increased ratio of background to supernova images, with rate of decay as a variable'''
  SupernovaImages=np.load("drive/My Drive/LArTPCdata/SupernovaImagesS"+f'{1797}'+"L"+f'{1000}'+"R"+f'{r}'+".npy")
  SupernovaLabels=np.ones(len(SupernovaImages))
  SNtrain_i,SNtrain_l,SNval_i,SNval_l,SNtest_i,SNtest_l=split_images(SupernovaImages,SupernovaLabels)

  maxSN=np.amax(SupernovaImages)

  BackgroundImages=np.load("drive/My Drive/LArTPCdata/BackgroundImagesratio"+f'{r}'+".npy")
  BackgroundLabels=np.zeros(len(BackgroundImages))
  BGtrain_i,BGtrain_l,BGval_i,BGval_l,BGtest_i,BGtest_l=split_images(BackgroundImages,BackgroundLabels)

  #find the maximal value for normalisation of all data on the same scale
  maxBG=np.amax(BackgroundImages)
  if maxBG>maxSN:
    max=maxBG
  else:
    max=maxSN

  #preprocess data
  training_images=np.concatenate((SNtrain_i,BGtrain_i))
  training_labels=np.concatenate((SNtrain_l,BGtrain_l))
  training_images, training_labels= preprocess_images(training_images,training_labels,max)
  validation_images=np.concatenate((SNval_i,BGval_i))
  validation_labels=np.concatenate((SNval_l,BGval_l))
  validation_images, validation_labels= preprocess_images(validation_images,validation_labels,max)
  test_images=np.concatenate((SNtest_i,BGtest_i))
  test_labels=np.concatenate((SNtest_l,BGtest_l))
  test_images, test_labels= preprocess_images(test_images,test_labels,max)

  #run the model
  model=binary_model()
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy",metrics=['accuracy'])
  model.fit(training_images, training_labels, epochs=EPOCHS, batch_size=BATCHES, validation_data=(validation_images, validation_labels),verbose=0,shuffle=True)

  #perform predictions
  prediction=model.predict(test_images,verbose=0)[:,0]
  labels_prediction=np.round(prediction)

  #0: negative (background), 1: positive (supernova)
  tn, fp, fn, tp = confusion_matrix(test_labels, labels_prediction, labels=[0,1]).ravel()
  acc_SN=tp/len(np.where(test_labels==1)[0])
  acc_BG=tn/len(np.where(test_labels==0)[0]) 
  uncert_SN=np.sqrt((acc_SN*(1-acc_SN))/len(np.where(test_labels==1)[0]))
  uncert_BG=np.sqrt((acc_BG*(1-acc_BG))/len(np.where(test_labels==0)[0]))

  return acc_SN,acc_BG,uncert_SN,uncert_BG

In [ ]:
rate=[5,10,15,20,25,30,35,40,45,50,60,70,80,90,100,120,150,175,200,250,300,400,500,600,700,800,900,1000]

acc_SN=[]
acc_BG=[]
uncertainty_SN=[]
uncertainty_BG=[]
N_fp=[]

#run the function for different rate values
for r in rate:
  acc1,acc2,uncert1,uncert2=run_CNN_rate_2(r)

  acc_SN.append(acc1)
  acc_BG.append(acc2)
  uncertainty_SN.append(uncert1)
  uncertainty_BG.append(uncert2)

  fp_percentage=1-acc2
  N_fp.append(r*1e6*fp_percentage)

In [ ]:
plt.figure()
#sns.regplot(x=rate, y=acc_SN,label="Supernova")
#sns.regplot(x=rate, y=acc_BG,label="Radiological")
plt.scatter(rate,acc_SN,label="Supernova",color='tab:blue')
plt.scatter(rate,acc_BG,label="Radiological",color="tab:orange")
plt.axhline(np.mean(acc_SN),color='tab:blue')
plt.axhline(np.mean(acc_BG),color="tab:orange")
plt.errorbar(rate,acc_SN, yerr=uncertainty_SN,color='tab:blue',fmt='o',capsize=3)
plt.errorbar(rate,acc_BG, yerr=uncertainty_BG,color='tab:orange',fmt='o',capsize=3)
plt.title("Testing Accuracy for Increased Radiological to Supernova Training Ratio")
plt.xlabel('Activity (μBq)')
plt.ylabel('Testing Accuracy')
plt.legend()
plt.tight_layout()
plt.savefig("drive/My Drive/Plots/Radiorates10ratio.png",bbox_inches = 'tight');

In [ ]:
def bg_rejection(x):
  '''calculate background rejection required to misclassify a maximum of 100 neutrinos'''
  return 1- 100/(x*1e6)

#plot relative error and runtime against N
color = 'tab:orange'
fig, ax1 =plt.subplots()
ax1.set_title("False Positives per second and required Background Accuracy for Increased Training Ratio")
ax1.set_xlabel("Activity (μBq)")
ax1.set_ylabel("Required Background Accuracy",color=color)
ax1.plot(rate,bg_rejection(np.asarray(rate)),color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_yscale('log')

ax2=ax1.twinx() # second axes with same x-axis

color = 'tab:blue'
ax2.set_ylabel("$N_{FP}$",color=color)
ax2.plot(rate,N_fp,"-o",color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_yscale('log')

plt.savefig("drive/My Drive/Plots/N_fp_ratio10.png",bbox_inches = 'tight')
plt.show();